## Movielens 데이터

1. 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 2000 년에 MovieLens에 가입한 6,040 명의 MovieLens 사용자가 약 3,900 편의 영화에 대한 1,000,209 개의 등급이 포함되어 있다.
2. 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
3. 별점을 시청횟수로 해석해서 생각하겠습니다.
4. 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

## 1) 데이터 준비와 전처리

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [4]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
del ratings['timestamp']

In [6]:
ratings.head()

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
# 검색을 쉽게하기 위해 제목 문자열을 소문자로 바꿔줍시다.
movies['title'] = movies['title'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


- movie_id와 제목을 매칭하기 위해 ratings DF와 movies DF 합치기(movie_id 기준으로)

In [9]:
ratings = ratings.join(movies.set_index('movie_id'), on='movie_id')
ratings.head()

,user_id,movie_id,counts,title,genre
0,1,1193,5,one flew over the cuckoo's nest (1975),Drama
1,1,661,3,james and the giant peach (1996),Animation|Children's|Musical
2,1,914,3,my fair lady (1964),Musical|Romance
3,1,3408,4,erin brockovich (2000),Drama
4,1,2355,5,"bug's life, a (1998)",Animation|Children's|Comedy


## 2) 분석해 봅시다.
- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [10]:
print('유니크한 영화 id : ', ratings['movie_id'].nunique())
print('유니크한 사용자 수: ', ratings['user_id'].nunique())

유니크한 영화 id :  3628
유니크한 사용자 수:  6039


In [11]:
# 인기 있는 영화 30개(인기순)
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [12]:
# 유저별 평균 영화 관람 횟수에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [13]:
movies[movies['title'].str.lower().str.contains('taxi', regex=False)]

,movie_id,title,genre
109,111,taxi driver (1976),Drama|Thriller


- 아이언맨 X, 스파이더맨 X, 분노의 질주 X, 해리포터 X, 아바타 X, 택시 X,,,, 뭐 있는게 없음 그냥 60개 보고 그나마 좋아하는걸로 고르기로 결정

In [14]:
movie_count.sort_values(ascending=False).head(60)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

## 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [15]:
# 내가 좋아하는 영화 리스트 5개
my_favorite = ['ghostbusters (1984)' , 'terminator 2: judgment day (1991)' ,'men in black (1997)' ,'armageddon (1998)' ,'Mission: Impossible (1996)']

# 'G0'이라는 user_id가 위 영화 리스트를 5회씩 봤다고 가정하겠습니다.
my_watchlist = pd.DataFrame({'user_id': ['6041']*5, 'title': my_favorite, 'counts':[5]*5})
my_watchlist

if not ratings.isin({'user_id':['6041']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_watchlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,title,genre
1000203,6040,1090.0,3,platoon (1986),Drama|War
1000205,6040,1094.0,5,"crying game, the (1992)",Drama|Romance|War
1000206,6040,562.0,5,welcome to the dollhouse (1995),Comedy|Drama
1000207,6040,1096.0,4,sophie's choice (1982),Drama
1000208,6040,1097.0,4,e.t. the extra-terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
0,6041,NaN,5,ghostbusters (1984),NaN
1,6041,NaN,5,terminator 2: judgment day (1991),NaN
2,6041,NaN,5,men in black (1997),NaN
3,6041,NaN,5,armageddon (1998),NaN
4,6041,NaN,5,Mission: Impossible (1996),NaN


필요없는 장르 컬럼 제거

In [16]:
del ratings['genre']

In [17]:
ratings.tail(10)  

,user_id,movie_id,counts,title
1000203,6040,1090.0,3,platoon (1986)
1000205,6040,1094.0,5,"crying game, the (1992)"
1000206,6040,562.0,5,welcome to the dollhouse (1995)
1000207,6040,1096.0,4,sophie's choice (1982)
1000208,6040,1097.0,4,e.t. the extra-terrestrial (1982)
0,6041,NaN,5,ghostbusters (1984)
1,6041,NaN,5,terminator 2: judgment day (1991)
2,6041,NaN,5,men in black (1997)
3,6041,NaN,5,armageddon (1998)
4,6041,NaN,5,Mission: Impossible (1996)


## 4) CSR matrix를 직접 만들어 봅시다.

In [18]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [20]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['6041'])    # 마지막 user_id 6040
print(movie_to_idx['Mission: Impossible (1996)'])

6039
3628


In [21]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_ratings = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_ratings) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_ratings   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_ratings = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_ratings) == len(ratings):
    print('movie column indexing OK!!')
    ratings['title'] = temp_movie_ratings
else:
    print('movie column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,counts,title
0,0,1193.0,5,0
1,0,661.0,3,1
2,0,914.0,3,2
3,0,3408.0,4,3
4,0,2355.0,5,4
...,...,...,...,...
0,6039,NaN,5,243
1,6039,NaN,5,92
2,6039,NaN,5,175
3,6039,NaN,5,125


In [22]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

# csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
csr_ratings = csr_matrix((ratings.counts, (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_ratings

<6040x3629 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [23]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [24]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [25]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_ratings_transpose = csr_ratings.T
csr_ratings_transpose

<3629x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [26]:
# 모델 훈련
als_model.fit(csr_ratings_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [38]:
g0, terminator = user_to_idx['6041'], movie_to_idx['terminator 2: judgment day (1991)']
g0_vector, terminator_vector = als_model.user_factors[g0], als_model.item_factors[terminator]

내 벡터

In [39]:
g0_vector

array([-1.13045466e+00,  3.36044848e-01, -6.34414284e-03,  1.06881785e+00,
        2.43759319e-01, -6.56816065e-01,  3.66169423e-01, -3.86433661e-01,
        3.31121385e-02,  5.69152832e-01,  4.50221390e-01,  8.21998179e-01,
        2.92045847e-02,  5.73734581e-01,  9.07566994e-02, -5.64319313e-01,
        8.14081192e-01, -1.84549138e-01,  3.80686373e-01,  1.03258774e-01,
        6.20011866e-01, -3.07088137e-01, -2.04382449e-01,  1.31323740e-01,
       -6.19629085e-01, -2.08661079e-01, -3.42124254e-01,  2.72987515e-01,
       -2.22394228e-01,  3.11980307e-01,  7.62550235e-01,  4.56062436e-01,
        1.55056655e-01,  1.47507057e-01,  3.83583546e-01, -3.28584045e-01,
        4.02295411e-01,  1.07392393e-01, -6.31627589e-02, -1.05920807e-01,
       -6.34892881e-01,  3.56768936e-01,  2.33177543e-01, -3.26399565e-01,
       -4.73463058e-01,  7.93932751e-02,  2.97170848e-01,  1.45161971e-01,
       -6.35525107e-01,  9.25920069e-01,  5.90335250e-01,  8.13527107e-01,
       -6.80569410e-01, -

영화 벡터

In [40]:
terminator_vector

array([-3.5906684e-02,  1.3015064e-02, -1.7189674e-02,  3.8089775e-02,
        1.9369405e-02, -3.1858914e-02,  1.6861770e-02,  1.4873260e-02,
        1.0842740e-02,  2.6163988e-02,  2.3507208e-02,  1.9068863e-02,
        6.4991612e-04,  2.3497393e-02, -1.3981261e-02, -1.4723687e-02,
       -3.1412201e-04,  1.3673508e-02,  8.1117721e-03, -2.1591891e-02,
        2.2655560e-02,  5.9282662e-05,  4.2075183e-02,  1.8965701e-02,
       -3.0222781e-02, -9.7178603e-03, -2.6140552e-02,  1.7273512e-02,
       -3.4011854e-03,  7.2196764e-03,  1.7486267e-02,  1.2610292e-02,
        5.2277171e-03,  2.3212079e-02, -7.5251982e-04,  6.7733335e-03,
        1.4778889e-02, -2.5085465e-03,  1.1311338e-02, -1.3581491e-02,
        6.3450346e-03,  6.5697278e-03,  8.7331552e-03, -9.1691026e-03,
        1.3574102e-02,  1.7461915e-02,  4.5584608e-03,  3.1773172e-02,
       -1.5795350e-03,  1.2871606e-02,  1.1749359e-02,  3.4890234e-02,
       -1.8354764e-02,  6.8871519e-03, -1.0360634e-03,  3.3301447e-02,
      

In [41]:
# 내 벡터와 영화의 벡터를 내적하는 코드
np.dot(g0_vector, terminator_vector)

0.55005336

선호도가 높게 나옴

In [43]:
# 다른 영화 넣어보기 
babe = movie_to_idx['babe (1995)']
babe_vector = als_model.item_factors[babe]
np.dot(g0_vector, babe_vector)

0.0119178435

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [45]:
favorite_movie = 'men in black (1997)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(175, 1.0),
 (107, 0.8327422),
 (92, 0.6113375),
 (62, 0.6017021),
 (150, 0.51635385),
 (124, 0.50655836),
 (82, 0.4864173),
 (145, 0.47562826),
 (3466, 0.4294864),
 (375, 0.41373035),
 (542, 0.3951131),
 (138, 0.38805336),
 (3628, 0.38127974),
 (320, 0.3799759),
 (1045, 0.37352487)]

In [46]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['men in black (1997)',
 'jurassic park (1993)',
 'terminator 2: judgment day (1991)',
 'total recall (1990)',
 'independence day (id4) (1996)',
 'matrix, the (1999)',
 'lost world: jurassic park, the (1997)',
 'fifth element, the (1997)',
 'schlafes bruder (brother of sleep) (1995)',
 'face/off (1997)',
 'rocky horror picture show, the (1975)',
 'true lies (1994)',
 'Mission: Impossible (1996)',
 'bob roberts (1992)',
 'grumpy old men (1993)']

In [47]:
#몇 번 더 반복해서 확인하기 위해 위의 코드를 함수로 만들고 확인
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [48]:
get_similar_movie('matrix, the (1999)')

['matrix, the (1999)',
 'terminator 2: judgment day (1991)',
 'total recall (1990)',
 'fugitive, the (1993)',
 'jurassic park (1993)',
 'fifth element, the (1997)',
 'terminator, the (1984)',
 'men in black (1997)',
 'face/off (1997)',
 'hunt for red october, the (1990)']

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [49]:
user = user_to_idx['6041']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_ratings, N=20, filter_already_liked_items=True)
movie_recommended

[(107, 0.56282616),
 (124, 0.41343838),
 (62, 0.37931755),
 (82, 0.36245424),
 (87, 0.32089144),
 (22, 0.3025416),
 (150, 0.28162336),
 (375, 0.27196744),
 (141, 0.24166279),
 (15, 0.23981026),
 (38, 0.23609294),
 (145, 0.23431307),
 (544, 0.2257635),
 (670, 0.220226),
 (60, 0.21590419),
 (48, 0.20466095),
 (200, 0.1882922),
 (80, 0.18252699),
 (201, 0.16998476),
 (456, 0.1681622)]

In [50]:
[idx_to_movie[i[0]] for i in movie_recommended]

['jurassic park (1993)',
 'matrix, the (1999)',
 'total recall (1990)',
 'lost world: jurassic park, the (1997)',
 'braveheart (1995)',
 'back to the future (1985)',
 'independence day (id4) (1996)',
 'face/off (1997)',
 'fugitive, the (1993)',
 'airplane! (1980)',
 'sixth sense, the (1999)',
 'fifth element, the (1997)',
 'stargate (1994)',
 'galaxy quest (1999)',
 'star wars: episode i - the phantom menace (1999)',
 'saving private ryan (1998)',
 'terminator, the (1984)',
 'stand by me (1986)',
 'who framed roger rabbit? (1988)',
 'arachnophobia (1990)']

AlternatingLeastSquares 클래스에 구현된 explain 메서드를 사용하면 제가 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있음

In [51]:
armageddon = movie_to_idx['armageddon (1998)']
explain = als_model.explain(user, csr_ratings, itemid=armageddon)

이 method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됩니다.)를 반환

In [53]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('armageddon (1998)', 0.2502652029778268),
 ('men in black (1997)', 0.04073637267696734),
 ('ghostbusters (1984)', 0.006202049422917994),
 ('Mission: Impossible (1996)', 0.0009980756429291537),
 ('terminator 2: judgment day (1991)', -0.0010243299774286528)]

기여도가 다 낮아짐

# 회고
- 나름 비슷하게 추천을 해주는 것 같은데 내가 좋아하는 영화는 찾는거 마다 없어서 아쉬웠다. 
- 이번 익스는 뭔가 한 코드 할 때마다 수정해야 하는 부분이 많아서 계속 오류가 났고 한땀 한땀 고쳐가면서 써내려가서 더 힘들었다. 
- 바쁠 때 꼭 일이 더 많이 터진다.